In [149]:
# -------------- #

import pandas as pd
import os
import datetime
import math

import load_data
from util_log import logger

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR,
    LAG_DICT,
    SLIDING_DICT
)

# from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error


from tqdm import tqdm, tqdm_notebook

ImportError: cannot import name 'FEATURE_DIR' from 'config' (/Users/liuyu/Desktop/g-forecast/config.py)

In [4]:
# valid period. '2017-07-31' to '2017-08-16'
SPLIT_DATE = datetime.date(2017,7, 31)

def add_weekday(df):
    df['weekday'] = df['date'].dt.weekday
    return df

def add_month(df):
    df['month'] = df['date'].dt.add_month
    return df





def item_info(df):
    items_df = pd.read_csv(RAW_DATA_DIR+'items.csv')
    df = df.merge(items_df, on='item_nbr')
    return df

def store_info(df):
    stores_df = pd.read_csv(RAW_DATA_DIR+'stores.csv')
    df = df.merge(items_df, on='store_nbr')
    return df

def transaction_info(df):
    transactions_df = pd.read_csv(RAW_DATA_DIR+'transactions.csv')
    df = df.merge(transactions_df, on = 'store_nbr')
    return df

def oil_info(df):
    oil_df = pd.read_csv(RAW_DATA_DIR+'oil.csv',parse_dates=['date'])
    df = df.merge(oil_df, on = 'date')




In [5]:
def lagging(df,
            col,
            lag_len):
    """ Generate lagging features.
    Args:
        df: a DataFrame, contains all columns in cols_lag
        len_lag: Int, the length of lagging features
        cols_lag: List of String, the columns needs to generate lagging features 

    Returns:
        df: a DataFrame with lagging features, for example 'Sales_lag_1'.
    """

    for lag in lag_len:
        df[col + '_lag_' + str(lag)] = df[col].shift(periods=lag)
        
    return df

In [132]:
logger.debug('Get raw data')
train_df, test_df = load_data.main()

Get raw data


In [38]:
def lagging(df,
            col,
            lag_len):
    """ Generate lagging features.
    Args:
        df: a DataFrame, contains all columns in cols_lag
        len_lag: Int, the length of lagging features
        cols_lag: List of String, the columns needs to generate lagging features 

    Returns:
        df: a DataFrame with lagging features, for example 'Sales_lag_1'.
    """

    for lag in lag_len:
        df[col + '_lag_' + str(lag)] = df[col].shift(periods=lag)
        
    return df

In [39]:
def sliding_window(df,
                   sliding_len,
                   col
                   ):
    """ Generate sliding window features.
    Args:
        df: a DataFrame, contains all columns in cols_lag
        len_sliding_window: Int, the length of sliding window, >2 
        cols_sliding: List of String, the columns needs to generate sliding window features 

    Returns:
        df: a DataFrame with sliding window features, for example 'unit_sales_mean_2'.
    """

    def roll(data, roll_size, col):
        roll = data[col].shift(1).rolling(roll_size)
        data[col + '_mean_' + str(roll_size)] = roll.mean()
        data[col + '_std_' + str(roll_size)] = roll.std()
#         data[col + '_max_' + str(roll_size)] = roll.max()
#         data[col + '_min_' + str(roll_size)] = roll.min()
        return data

    for roll_size in sliding_len:
        df = roll(df, roll_size, col)
    df = df.dropna()
    return df

In [119]:
def filling_missing_value(store_df,
                          item_df,
                          start_date,
                          end_date):
    """ filling missing value.
    Args:
        item_df: a DataFrame, contains only one store data with cols: 'Sales'

    Returns:
        df: a DataFrame. store_frame data filled
    """


    date_range = pd.date_range(start_date, end_date)
    item_nbr = item_df.item_nbr.unique()[0]
    store_nbr = item_df.store_nbr.unique()[0]
    unit_sales = 0
    for date in date_range:
        if item_df[item_df['date'] == date].shape[0] != 0:
            continue
        try:
            onpromotion = store_df[store_df['date'] == date].dropna().unique()[0]
        except:
            onpromotion = False
        item_df.loc[item_df.shape[0]] = [date, store_nbr, item_nbr, unit_sales, onpromotion]
    item_df = item_df.sort_values('date')
    item_df = item_df.reset_index(drop=True)
    return item_df



In [ ]:
storeId_list = train_df['store_nbr'].unique()
# logger.debug('Filling missing value, lag, sliding window')
for storeId in tqdm(storeId_list, desc = 'store loop'):
    if storeId==25:
        continue
    store_frame = train_df[train_df['store_nbr'] == storeId]
    item_list = store_frame.item_nbr.unique()
    count = 0
    logger.debug(count)
    for itemId in tqdm(item_list,mininterval = 300, desc = 'item loop'):
        item_frame = store_frame[store_frame['item_nbr']==itemId]
        item_frame = item_frame.sort_values(by=['date'])
        item_frame = item_frame.reset_index(drop=True)

        # missing 
        item_frame = filling_missing_value(store_frame, item_frame, '2017-05-01', '2017-08-15')        
        # outlier filter
        
        for col in LAG_DICT:
            lag_len = LAG_DICT[col]
            item_frame = lagging(item_frame, col, lag_len)

        for col in SLIDING_DICT:
            sliding_len = SLIDING_DICT[col]
            item_frame = sliding_window(item_frame,
                                     sliding_len,
                                     col) 

        if count == 0:
            df = item_frame
        else:
            df = pd.concat([df, item_frame])
        count += 1

    df.to_csv(FEATUER_DIR+'{}_lag_sales.csv'.format(storeId), index=False)
    del df













store loop:   0%|          | 0/54 [00:00<?, ?it/s]0













item loop:   0%|          | 0/3540 [00:00<?, ?it/s]












item loop:  27%|██▋       | 955/3540 [05:00<13:32,  3.18it/s]












item loop:  27%|██▋       | 955/3540 [05:14<13:32,  3.18it/s]












item loop:  48%|████▊     | 1691/3540 [10:00<10:33,  2.92it/s]












item loop:  48%|████▊     | 1691/3540 [10:14<10:33,  2.92it/s]












item loop:  63%|██████▎   | 2228/3540 [15:01<08:54,  2.45it/s]












item loop:  63%|██████▎   | 2228/3540 [15:14<08:54,  2.45it/s]












item loop:  76%|███████▌  | 2696/3540 [20:01<06:43,  2.09it/s]












item loop:  76%|███████▌  | 2696/3540 [20:14<06:43,  2.09it/s]












item loop:  88%|████████▊ | 3121/3540 [25:01<03:48,  1.83it/s]












item loop:  88%|████████▊ | 3121/3540 [25:14<03:48,  1.83it/s]












item loop: 100%|██████████| 3540/3540 [29:41<00:00,  1.99it/s]











store loop:   4%|▎         | 2/54 [30

In [ ]:
feature

In [153]:
df.to_csv(FEATUER_DIR+'25_lag_sales.csv')

In [14]:
train_df = train_df.dropna()

In [15]:
def train_valid_split(df):
    train_df = df[df['date']<SPLIT_DATE]
    valid_df = df[df['date']>=SPLIT_DATE]
    return train_df, valid_df

In [16]:
train, valid = train_valid_split(train_df)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  
/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
def feature_label(df):

    features = df.drop(columns = ['date', 'store_nbr', 'item_nbr', 'unit_sales']).values
    labels = df['unit_sales'].values
    return features, labels

In [23]:
train_X, train_y = feature_label(train)
valid_X, valid_y = feature_label(valid)

In [1]:
train_X = [[1],[2],[3],[4]]

train_y=[1,0,0,1]

In [4]:
# model =LGBMRegressor(
#                 colsample_bytree = 0.8,
#                 learning_rate= 0.08,  # [0.08, 0.1]
#                 max_depth=6,
#                 num_leaves= 30,
#                 min_child_weight= 5,
#                 n_estimators= 200,  # [300, 500],
# #                 nthread = 4,
#                 seed= 1337,
#                 silent= 1,
#                 subsample= 0.8
#                 )


In [25]:
# model = CatBoostRegressor(iterations= 50,
#                 depth = 6,
#                 learning_rate= 0.5,
#                 loss_function ='RMSE',
#                          verbose = 0)

In [31]:
model =XGBRegressor()

In [32]:
model.fit(train_X, train_y)

[16:25:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [33]:
pred = model.predict(valid_X)

In [34]:
mean_squared_error(valid_y, pred)

1.3343208463934615

In [ ]:
# Feature importance

feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')